In [1]:
# -*- coding: utf-8 -*-

# ケーブル連携方法
- 首：基盤 [::: 空白]の下側を左からBRY
- 髪の毛とおかん：ケーブル同士の接続は、黒と茶色

# ライブラリインポート

In [2]:
# import libaries
import pyaudio #音声wav入出力
import pygame #mp3再生
from mutagen.mp3 import MP3 #ファイル再生時間取得
import wave #音声波長データ
import threading
import time
import numpy as np
import serial

#import sys
#sys.path.append("/Users/kimiaki/.pyenv/versions/2.7.13/lib/python2.7/site-packages")
#import serial

# 関数作成

In [3]:
# 関数作成

# 首の初期化
def initialize_neck(ser):
    ser.write(b'j')
    ser.write(b'D')
    
# 首を動かす
def move_neck(ser, neck_count = 0):
    ser.write('{}'.format(neck_level[neck_count]).encode())

# 髪の毛を揺らす
def move_hair(ser):
    ser.write(b'B')
    time.sleep(0.2)
    ser.write(b'D')
    time.sleep(0.2)
    ser.write(b'B')
    time.sleep(0.2)
    ser.write(b'D')
    
# 髪の毛を飛ばす
def shot_hair(ser):
    ser.write(b'J')
    shout_voice1("voice/飛ぶ音.mp3")
    #time.sleep(0.2)
    

# 指定の音声ファイルを出力(wav)
def shout_voice(filename = ""):
    print("音声開始")
    CHUNK = 1024
    if filename == "":
        filename = "voice/bakamon.wav"
    wf = wave.open(filename, 'rb')
    p = pyaudio.PyAudio()
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True)
    # 1024個読み取り
    data = wf.readframes(CHUNK)
    while data != '':
        stream.write(data)          
        data = wf.readframes(CHUNK) 

    stream.stop_stream()
    stream.close()
    p.terminate()
    print("音声終了")

# 指定の音声ファイルを出力(mp3)
def shout_voice1(filename = ""):
    # ファイル再生時間取得
    audio = MP3(filename)
    # 再生
    pygame.mixer.init()
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play(1)
    time.sleep(audio.info.length + 0.5)
    #pygame.mixer.music.set_volume(3.5) # ボリュームの設定
    #pygame.mixer.music.queue('voice/時間オーバー.mp3') # 次のファイル設定


# ゲーム終了
def stop_game(stream, p):
    stream.stop_stream()
    stream.close()
    p.terminate()

In [4]:
# 1秒間の入力音声データから、RMSで音量算出し、最大音量を取得する
def get_volume():
    
    CHUNK = 1024
    FORMAT = pyaudio.paInt16 # int16型
    CHANNELS = 1             # モノラル
    RATE = 16000             # 160.kHz
    

    RECORD_SECONDS = 1       # 1秒録音
    data_rms = []
    loopcounter = 0
    plottime = 1

    p = pyaudio.PyAudio()

    while True:
        stream = p.open(format = pyaudio.paInt16,
                       channels = 1,
                       rate = 16000,
                       input = True,
                       frames_per_buffer = 1024)
        all = []
        for i in range(0, RATE / CHUNK * RECORD_SECONDS):
            data = stream.read(CHUNK)
            all.append(data)

        data = ''.join(all)
        data = np.frombuffer(data, dtype = "int16")

        # numpyのarrayに変換
        np_data = np.array(data, dtype = np.float64)

        # downsampling
        overhang = len(np_data) % 100
        down_data = np_data[:-overhang]
        down_data = np.reshape(down_data, (len(down_data)/100, 100))
        down_data = np.average(down_data, 1)

        # RMS計算
        rms = np.sqrt(np.mean(np.square(down_data)))
        #print(rms)
        data_rms.append(rms)
        stream.close()
        loopcounter += 1
        if loopcounter == plottime:
            break
    return rms

# 動作確認

## 基本動作

In [87]:
ser = serial.Serial('/dev/tty.usbmodem14121', 9600)

In [88]:
initialize_neck(ser)

In [73]:
move_hair(ser)

In [58]:
shot_hair(ser)

In [59]:
initialize_neck(ser)

In [61]:
neck_level = {0: 'j', 1: 'i', 2: 'h', 3: 'g', 4: 'f', 5: 'e', 6: 'd', 7: 'c', 8: 'b', 9: 'a'}
move_neck(ser, 1)
#ser.write(b'')

## マイク音量の確認

In [75]:
ser = serial.Serial('/dev/tty.usbmodem14121', 9600)

In [90]:
for i in range(20):
    volume = get_volume()
    print(volume)

402.091474631
8353.74672593
8159.02635687
3179.78770973
12203.5949078
499.158589086


KeyboardInterrupt: 

## 出力音声の確認

In [83]:
shout_voice1("voice/成功.mp3")
shout_voice1("voice/時間オーバー.mp3")
shout_voice1("voice/飛ばしすぎ.mp3")
shout_voice1("voice/飛ぶ音.mp3")

In [84]:
import pygame
pygame.mixer.init()
pygame.mixer.music.load("voice/BGM.mp3")
pygame.mixer.music.play(1)
time.sleep(10)
pygame.mixer.music.stop()

In [86]:
pygame.mixer.music.stop()

# ゲームコード

In [103]:
CHUNK = 1024*2
RATE = 44100
p = pyaudio.PyAudio()

# 変数を初期設定
neck_count = 0 #首の回転
neck_level = {0: 'j', 1: 'i', 2: 'h', 3: 'g', 4: 'f', 5: 'e', 6: 'd', 7: 'c', 8: 'b', 9: 'a'}

stream = p.open(format = pyaudio.paInt16,
                channels = 1,
                rate = RATE,
                frames_per_buffer = CHUNK,
                input = True,
                output = True) # inputとoutputを同時にTrueにする

ser = serial.Serial('/dev/tty.usbmodem14121', 9600)

# モーター位置を初期化
initialize_neck(ser)

'''
# BGM開始
pygame.mixer.init()
pygame.mixer.music.load("voice/BGM.mp3")
pygame.mixer.music.play(1)
'''

time.sleep(2)

print("**********************************************************")
print("**********************  ゲーム開始！  **********************")
print("**********************************************************")


# ゲーム開始
check_count = neck_count
print("************[残りライフ：{} pt / 10 pt]************".format(str(9-neck_count)))

# 経過計測
start = time.time()

while stream.is_active():
    #print("首のカウント: {}".format(neck_count))
    if check_count != neck_count:
        print("************[残りライフ：{} pt / 10 pt]************".format(str(9-neck_count)))
        check_count = neck_count

    # 10秒ごとに必ず首を1つ回す
    end = time.time()
    elapse = end - start
    #print("経過時間: {}".format(elapse))
    if elapse > 7 or neck_count == 9:
        # 首を1つ回す
        neck_count += 1
        move_neck(ser, neck_count)
        move_hair(ser)
        time.sleep(1)
        if neck_count == 9:
            print("************「いい加減にバレた！」************")
            pygame.mixer.music.stop()
            shout_voice1("voice/時間オーバー.mp3")
            #time.sleep(1)
            #stop_game(stream, p)
            break
        start = end
    
    # 音量取得
    volume = get_volume()
    print("風量: {}".format(volume))

    if volume >= 5000 and volume <= 15000: #input<10:何もしない
        if volume < 10000: #弱すぎると、首を1つ回し、髪の毛を揺らす
            neck_count += 1
            move_neck(ser, neck_count) #首が動く
            move_hair(ser) #髪が揺れる
            time.sleep(1)
        elif volume < 10050: #ちょうど良いと、髪の毛飛ばし成功
            print("************「髪の毛飛ばし成功！」************")
            print(volume)
            pygame.mixer.music.stop()
            shot_hair(ser) #髪の毛を飛ばす
            shout_voice1("voice/成功.mp3") #成功の音
            shout_voice1("voice/驚き.mp3") #マ○オさんが驚く
            #time.sleep(1)
            #stop_game(stream, p)
            break
        else: #強すぎると、飛ぶが怒られる
            print("************「飛ぶが怒られる！」************")
            print(volume)
            pygame.mixer.music.stop()
            #shot_hair(ser) #髪の毛を飛ばす
            shout_voice1("voice/飛ばしすぎ.mp3") #怒られる
            #time.sleep(1)
            #stop_game(stream, p)
            break
    else:
        time.sleep(1)

        
time.sleep(1)
stop_game(stream, p)

print("**********************************************************")
print("**********************  Game Over！ **********************")
print("**********************************************************")

pygame.mixer.music.stop()

time.sleep(2)
initialize_neck(ser)
time.sleep(2)
ser.close()
print("\n\nゲーム終了")

**********************************************************
**********************  ゲーム開始！  **********************
**********************************************************
************[残りライフ：9 pt / 10 pt]************
風量: 735.386737056
風量: 7543.09494937
************[残りライフ：8 pt / 10 pt]************
風量: 12391.0606655
************「飛ぶが怒られる！」************
12391.0606655
**********************************************************
**********************  Game Over！ **********************
**********************************************************


ゲーム終了


# その他

## 音声保存(wav)

In [128]:
# 音声の保存

import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16 # int16型
CHANNELS = 2             # ステレオ
RATE = 44100             # 441.kHz
RECORD_SECONDS = 5      # 5秒録音
WAVE_OUTPUT_FILENAME = "voice/sample.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [131]:
# 確認
shout_voice("voice/sample.wav")

音声開始
音声終了


## 音声出力(wav)

In [94]:
# 音声の出力

import pyaudio
import wave

CHUNK = 1024
filename="voice/sample.wav"

wf = wave.open(filename, 'rb')

p = pyaudio.PyAudio()

stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                channels=wf.getnchannels(),
                rate=wf.getframerate(),
                output=True)

""" 
   format  : ストリームを読み書きするときのデータ型
   channels: ステレオかモノラルかの選択 1でモノラル 2でステレオ
   rate    : サンプル周波数
   output  : 出力モード

"""

# 1024個読み取り
data = wf.readframes(CHUNK)

while data != '':
    stream.write(data)          
    data = wf.readframes(CHUNK) 

stream.stop_stream()
stream.close()

p.terminate()

## 音声出力(mp3)

In [162]:
import pygame
import time

pygame.mixer.init()
pygame.mixer.music.load("voice/成功.mp3")
pygame.mixer.music.play(1)
time.sleep(2)
pygame.mixer.music.load("voice/飛ばしすぎ.mp3")
#pygame.mixer.music.set_volume(3.5) # ボリュームの設定
pygame.mixer.music.play(1)
time.sleep(5)
#pygame.mixer.music.queue('voice/時間オーバー.mp3')
#print(pygame.mixer.music.get_busy())
pygame.mixer.music.load("voice/時間オーバー.mp3")
pygame.mixer.music.play(1)

## 音量取得方法の検討

In [ ]:
# 1秒間の入力音声データから、RMSで音量算出し、最大音量を取得する
import pyaudio
import wave
import time
import numpy as np
import threading

CHUNK = 1024
FORMAT = pyaudio.paInt16 # int16型
CHANNELS = 1             # モノラル
RATE = 16000             # 160.kHz
RECORD_SECONDS = 1       # 1秒録音
data_rms = []
loopcounter = 0
plottime = 1

p = pyaudio.PyAudio()

while True:
    stream = p.open(format = FORMAT,
                   channels = CHANNELS,
                   rate = RATE,
                   input = True,
                   frames_per_buffer = CHUNK)
    all = []
    for i in range(0, RATE / CHUNK * RECORD_SECONDS):
        data = stream.read(CHUNK)
        all.append(data)
    
    data = ''.join(all)
    data = np.frombuffer(data, dtype = "int16")
    
    # numpyのarrayに変換
    np_data = np.array(data, dtype = np.float64)
    
    # downsampling
    overhang = len(np_data) % 100
    down_data = np_data[:-overhang]
    down_data = np.reshape(down_data, (len(down_data)/100, 100))
    down_data = np.average(down_data, 1)
    
    # RMS計算
    rms = np.sqrt(np.mean(np.square(down_data)))
    print(rms)
    data_rms.append(rms)
    stream.close()
    loopcounter += 1
    if loopcounter == plottime:
        break